# Pesquisa com Dados de Satélite (Satellite)

O banco de dados consiste nos valores multi-espectrais de pixels em vizinhanças 3x3 em uma imagem de satélite, e na classificação associada ao pixel central em cada vizinhança. O objetivo é prever esta classificação, dados os valores multi-espectrais.

Um quadro de imagens do Satélite Landsat com MSS (Multispectral Scanner System) consiste em quatro imagens digitais da mesma cena em diferentes bandas espectrais. Duas delas estão na região visível (correspondendo aproximadamente às regiões verde e vermelha do espectro visível) e duas no infravermelho (próximo). Cada pixel é uma palavra binária de 8 bits, com 0 correspondendo a preto e 255 a branco. A resolução espacial de um pixel é de cerca de 80m x 80m. Cada imagem contém 2340 x 3380 desses pixels. O banco de dados é uma subárea (minúscula) de uma cena, consistindo de 82 x 100 pixels. Cada linha de dados corresponde a uma vizinhança quadrada de pixels 3x3 completamente contida dentro da subárea 82x100. Cada linha contém os valores de pixel nas quatro bandas espectrais (convertidas em ASCII) de cada um dos 9 pixels na vizinhança de 3x3 e um número indicando o rótulo de classificação do pixel central.

As classes são: solo vermelho, colheita de algodão, solo cinza, solo cinza úmido, restolho de vegetação, solo cinza muito úmido.

Os dados estão em ordem aleatória e certas linhas de dados foram removidas, portanto você não pode reconstruir a imagem original desse conjunto de dados. Em cada linha de dados, os quatro valores espectrais para o pixel superior esquerdo são dados primeiro, seguidos pelos quatro valores espectrais para o pixel superior central e, em seguida, para o pixel superior direito, e assim por diante, com os pixels lidos em sequência, da esquerda para a direita e de cima para baixo. Assim, os quatro valores espectrais para o pixel central são dados pelos atributos 17, 18, 19 e 20. Se você quiser, pode usar apenas esses quatro atributos, ignorando os outros. Isso evita o problema que surge quando uma vizinhança 3x3 atravessa um limite.

O banco de dados se encontra no pacote **mlbench** e é completo (não possui dados faltantes).


## Tarefas

#### 1. Treine modelos RandomForest, SVM e RNA para predição destes dados. 

In [7]:
install.packages('mlbench')

Warning message in install.packages("mlbench"):
“installation of package ‘mlbench’ had non-zero exit status”Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [2]:
library('mlbench')

- Analisando os dados

In [60]:
dataset <- Satellite[c(17:20, 37)]
head(dataset)

x.17,x.18,x.19,x.20,classes
92,112,118,85,grey soil
84,103,104,81,grey soil
84,99,104,78,grey soil
84,99,104,81,grey soil
76,99,104,81,grey soil
76,99,108,85,grey soil


In [61]:
frequencia_abs <- table(dataset$classes)
frequencia_rel <- prop.table(frequencia_abs)
data.frame('Classe'= names(frequencia_abs),
          'Frequência'= as.vector(frequencia_abs),
          'Relativa'= as.vector(frequencia_rel),
          'Percentagem'= as.vector(frequencia_rel) * 100)

Classe,Frequência,Relativa,Percentagem
red soil,1533,0.2382284,23.82284
cotton crop,703,0.1092463,10.92463
grey soil,1358,0.2110334,21.10334
damp grey soil,626,0.0972805,9.72805
vegetation stubble,707,0.1098679,10.98679
very damp grey soil,1508,0.2343434,23.43434


- Separando 80% dos dados para treino e 20% para teste.

In [62]:
library('caret')
indexes <- createDataPartition(dataset$classes, p = 0.80, list = FALSE)
train_data <- dataset[indexes,]
test_data <- dataset[-indexes,]

- Treinando os modelos

In [79]:
rf_model <- train(classes ~ ., data = train_data, method = 'rf')
svm_model <- train(classes ~ ., data = train_data, method = 'svmRadial')
rna_model <- train(classes ~ ., data = train_data, method = 'nnet', trace = FALSE)

- Aplicando os modelos no dataset de testes

In [81]:
rf_predictions = predict(rf_model, test_data)
svm_predictions <- predict(svm_model, test_data)
rna_predictions <- predict(nn_model, test_data)

----

#### 2. Escolha o melhor modelo com base em suas matrizes de confusão. 

In [123]:
cm_rf <- confusionMatrix(rf_predictions, test_data$classes)
cm_rf

Confusion Matrix and Statistics

                     Reference
Prediction            red soil cotton crop grey soil damp grey soil
  red soil                 294           2         2              1
  cotton crop                1         127         0              0
  grey soil                  5           0       249             24
  damp grey soil             0           1        15             53
  vegetation stubble         5           8         0              1
  very damp grey soil        1           2         5             46
                     Reference
Prediction            vegetation stubble very damp grey soil
  red soil                            13                   0
  cotton crop                          3                   1
  grey soil                            1                   6
  damp grey soil                       1                  31
  vegetation stubble                 110                   8
  very damp grey soil                 13                 255

O

In [124]:
cm_svm <- confusionMatrix(svm_predictions, test_data$classes)
cm_svm

Confusion Matrix and Statistics

                     Reference
Prediction            red soil cotton crop grey soil damp grey soil
  red soil                 296           1         0              0
  cotton crop                0         127         0              1
  grey soil                  4           0       260             27
  damp grey soil             0           0        10             62
  vegetation stubble         5           9         0              1
  very damp grey soil        1           3         1             34
                     Reference
Prediction            vegetation stubble very damp grey soil
  red soil                            13                   1
  cotton crop                          0                   1
  grey soil                            0                   6
  damp grey soil                       0                  29
  vegetation stubble                 111                   4
  very damp grey soil                 17                 260

O

In [125]:
cm_rna <- confusionMatrix(rna_predictions, test_data$classes)
cm_rna

Confusion Matrix and Statistics

                     Reference
Prediction            red soil cotton crop grey soil damp grey soil
  red soil                 291           2         1              0
  cotton crop                0         128         0              0
  grey soil                  7           0       239             34
  damp grey soil             0           0         7              1
  vegetation stubble         7           7         0              0
  very damp grey soil        1           3        24             90
                     Reference
Prediction            vegetation stubble very damp grey soil
  red soil                            20                   1
  cotton crop                         19                   0
  grey soil                            1                  13
  damp grey soil                       0                   0
  vegetation stubble                  52                   2
  very damp grey soil                 49                 285

O

-----

#### 3. Treine o modelo final com todos os dados e faça a predição na base completa.

In [108]:
print(svm_model)

Support Vector Machines with Radial Basis Function Kernel 

5151 samples
   4 predictor
   6 classes: 'red soil', 'cotton crop', 'grey soil', 'damp grey soil', 'vegetation stubble', 'very damp grey soil' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 5151, 5151, 5151, 5151, 5151, 5151, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.8537440  0.8188487
  0.50  0.8572141  0.8232122
  1.00  0.8593428  0.8258647

Tuning parameter 'sigma' was held constant at a value of 0.8280511
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.8280511 and C = 1.


In [117]:
library('kernlab')

final_model <- ksvm(type = 'C-svc', classes ~ ., data = dataset, 
                    kernel = 'rbfdot', C = 1.0, kpar = list(sigma = 0.8280511))

In [120]:
final_model_predictions <- predict(final_model, dataset)

------

#### 4. Analise o resultado.

In [127]:
data.frame('Modelo' = c('RF', 'SVM', 'RNA'), 'Acurácia' =  c(
    cm_rf$overall['Accuracy'],
    cm_svm$overall['Accuracy'],
    cm_rna$overall['Accuracy'] ))

Modelo,Acurácia
RF,0.8473520
SVM,0.8691589
RNA,0.7757009


A partir das predicões feitas é possível afirmar que o modelo baseado em SVM se destaca por possuir maior acurácia (0.8691589).

Já o modelo final, gerado a partir de um SVM e considerando todos os dados, alcançou 0.8671 de acurácia conforme a matriz de confusão abaixo.

In [129]:
cm_final_model <- confusionMatrix(final_model_predictions, dataset$classes)
cm_final_model

Confusion Matrix and Statistics

                     Reference
Prediction            red soil cotton crop grey soil damp grey soil
  red soil                1499           2        10              5
  cotton crop                2         636         0              1
  grey soil                 19           0      1303            160
  damp grey soil             0           6        40            311
  vegetation stubble        12          49         0              4
  very damp grey soil        1          10         5            145
                     Reference
Prediction            vegetation stubble very damp grey soil
  red soil                            48                   0
  cotton crop                          9                   2
  grey soil                            3                  48
  damp grey soil                       4                 169
  vegetation stubble                 571                  29
  very damp grey soil                 72                1260

O

-----

#### 5. Salve este modelo final 

In [130]:
saveRDS(final_model, "satellite_svm.rds")